# Notebook for visualizing trajectories for both cities

In [ ]:
import os
import sys

def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder (e.g., 'masteroppgave') by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

# Find and append 'masteroppgave' to sys.path dynamically
project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")

In [20]:
import folium
from folium.plugins import FeatureGroupSubGroup
import random
from constants import  *
from utils.helpers import file_handler
import pprint

# Setup

In [21]:
#City to use (used for filepath)
CITY = "rome"
TRAJECTORIES = ["BXV", "BXZ"]

In [22]:
def build_list_of_files (trajectory_names: list[str], prefix: str):
    list_of_files = []
    for name in trajectory_names:
        list_of_files.append(f"{prefix}_{name}.txt")
    return list_of_files

In [23]:
if CITY == "rome":
    OUTPUT_FOLDER = f"../dataset/rome/output/"
    MAX_LON = 12.53
    MIN_LON = 12.44
    MAX_LAT = 41.93
    MIN_LAT = 41.88
    file_list = build_list_of_files(TRAJECTORIES, "R")
    trajectories = file_handler.load_trajectory_files(file_list, OUTPUT_FOLDER) ## Reads a list of trajectory files, return a dictionary with the filename as key and coordinates as values

if CITY == "porto":
    OUTPUT_FOLDER = f"../dataset/porto/output/"
    MAX_LON = -8.57
    MIN_LON = -8.66
    MAX_LAT = 41.19
    MIN_LAT = 41.14 
    file_list = build_list_of_files(TRAJECTORIES, "P")
    trajectories = file_handler.load_trajectory_files(file_list, OUTPUT_FOLDER) ## Reads a list of trajectory files, return a dictionary with the filename as key and coordinates as values


# Visualize trajectories

In [ ]:
def visualize_trajectories_with_filtering(trajectories):
    """
    Visualizes multiple trajectories on a Folium map with checkboxes to filter them.

    Parameters:
    - trajectories (dict): Dictionary where keys are trajectory names, and values are lists of (lat, lon) coordinates.

    Returns:
    - A Folium map object.
    """

    if not trajectories:
        raise ValueError("Trajectory dictionary is empty!")

    center_lat = (MIN_LAT + MAX_LAT) / 2
    center_lon = (MIN_LON + MAX_LON) / 2

    # Initialize Folium map
    map_trajectories = folium.Map(location=[center_lat, center_lon], zoom_start=13)

    # Add bounding box for Rome coordinates
    folium.Rectangle(
        bounds=[(MIN_LAT, MIN_LON), (MAX_LAT, MAX_LON)],
        color="black",
        weight=2,
        fill=True,
        fill_opacity=0.1,
        popup="Rome Bounding Box"
    ).add_to(map_trajectories)

    # Define a set of distinct colors for trajectories
    traj_colors = [
        "red", "blue", "green", "purple", "orange", "pink", "brown", "cyan", "magenta", "yellow", "lime"
    ]
    random.shuffle(traj_colors)  # Shuffle colors for randomness

    # Add a FeatureGroup for each trajectory to enable toggling
    for idx, (traj_name, coords) in enumerate(trajectories.items()):
        traj_color = traj_colors[idx % len(traj_colors)]  # Assign a unique color
        traj_layer = folium.FeatureGroup(name=f"Trajectory: {traj_name}")  # Create a feature group

        # Add trajectory line to its feature group
        folium.PolyLine(
            coords,
            color=traj_color,
            weight=6.5,
            opacity=1,
            popup=f"Trajectory: {traj_name}",
        ).add_to(traj_layer)

        # Add markers for each point along the trajectory
        for lat, lon in coords:
            folium.CircleMarker(
                location=(lat, lon),
                radius=1,  # Bigger markers
                color="black",
                fill=True,
                fill_color="red",
                fill_opacity=1,
                popup=f"Point: ({lat:.5f}, {lon:.5f})"
            ).add_to(traj_layer)

        traj_layer.add_to(map_trajectories)  # Add trajectory layer to map

    # Add LayerControl to enable checkboxes
    folium.LayerControl(collapsed=False).add_to(map_trajectories)

    return map_trajectories

# Generate the map with filterable checkboxes
map_trajectories = visualize_trajectories_with_filtering(trajectories)

# # Save the map
# map_trajectories.save("trajectories_rome.html")

# print("Map with filterable trajectories saved as trajectories_rome_filtered.html")

map_trajectories